# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:

# Study data files
cities_path = "Data/output_file"

# Read the mouse data and the study results
cities = pd.read_csv(cities_path)

cities_df = pd.DataFrame(cities)

cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,new norfolk,-42.7826,147.0587,18.14,38,94,0.89,AU,1643788104
1,saldanha,-33.0117,17.9442,19.03,87,100,6.40,ZA,1643788104
2,nanguneri,8.4933,77.6581,31.56,43,4,4.11,IN,1643788104
3,talnakh,69.4865,88.3972,-18.22,96,98,3.12,RU,1643788059
4,rikitea,-23.1203,-134.9692,25.74,67,100,0.91,PF,1643788105
...,...,...,...,...,...,...,...,...,...
564,biak,-0.9131,122.8766,27.67,65,100,0.60,ID,1643788131
565,dawson,31.7735,-84.4466,11.83,90,91,5.23,US,1643788208
566,syedove,47.0768,38.1560,2.22,93,100,2.60,UA,1643788208
567,soledade,-28.8183,-52.5103,16.24,84,0,2.22,BR,1643788208


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

In [10]:
 # Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
# Narrow down the cities to fit weather conditions

# max temperature
temp_df = cities_df[(cities_df["Max Temp"] > 30) & (cities_df["Max Temp"] < 50)]

# cloudiness
cloudiness_df = temp_df[temp_df["Cloudiness"] < 10]

# Wind speed less than 10 mph
wind_speed_df = cloudiness_df[cloudiness_df["Wind Speed"] < 10]

# Drop any rows will null values
wind_speed_df = wind_speed_df.dropna()
wind_speed_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,nanguneri,8.4933,77.6581,31.56,43,4,4.11,IN,1643788104
8,port elizabeth,-33.9180,25.5701,31.16,58,0,1.03,ZA,1643787981
43,busselton,-33.6500,115.3333,32.25,28,0,2.01,AU,1643788020
140,east london,-33.0153,27.9116,31.52,58,0,1.03,ZA,1643787987
203,geraldton,-28.7667,114.6000,42.23,13,0,6.69,AU,1643788016
281,beloha,-25.1667,45.0500,33.16,30,0,5.43,MG,1643788158
364,matara,5.9485,80.5353,30.72,50,8,4.57,LK,1643788172
421,ca mau,9.1769,105.1500,32.60,44,5,6.65,VN,1643787903


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [119]:
hotel_df = wind_speed_df
hotel_df["Hotel Name"] = ""

hotel_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,nanguneri,8.4933,77.6581,31.56,43,4,4.11,IN,1643788104,
8,port elizabeth,-33.9180,25.5701,31.16,58,0,1.03,ZA,1643787981,
43,busselton,-33.6500,115.3333,32.25,28,0,2.01,AU,1643788020,
140,east london,-33.0153,27.9116,31.52,58,0,1.03,ZA,1643787987,
203,geraldton,-28.7667,114.6000,42.23,13,0,6.69,AU,1643788016,
281,beloha,-25.1667,45.0500,33.16,30,0,5.43,MG,1643788158,
364,matara,5.9485,80.5353,30.72,50,8,4.57,LK,1643788172,
421,ca mau,9.1769,105.1500,32.60,44,5,6.65,VN,1643787903,


In [112]:

target_type = "hotel"
location = f"{lat},{lng}"
radius = 5000

params = {  
    "radius": radius,
    "types": target_type,
    "key": g_key,
    "keyword": "hotels"
}


for index, row in hotel_df.iterrows():
    
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        lat = row["Latitude"]
        lng = row["Longitude"]
    
        params['location'] = f'{lat},{lng}'
  
        response = requests.get(base_url, params=params)
        hotel_search = response.json()
#         print(hotel_search)

        try:
            hotel_df.loc[index, "Hotel Name"] = (hotel_search["results"][0]["name"])
        except: 
            hotel_df.loc[index, "Hotel Name"] = "NA"
            pass
        
hotel_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,nanguneri,8.4933,77.6581,31.56,43,4,4.11,IN,1643788104,NA
8,port elizabeth,-33.9180,25.5701,31.16,58,0,1.03,ZA,1643787981,Island Way Villa
43,busselton,-33.6500,115.3333,32.25,28,0,2.01,AU,1643788020,Bayview Geographe Resort
140,east london,-33.0153,27.9116,31.52,58,0,1.03,ZA,1643787987,Blue Lagoon Hotel
203,geraldton,-28.7667,114.6000,42.23,13,0,6.69,AU,1643788016,Ocean Centre Hotel
281,beloha,-25.1667,45.0500,33.16,30,0,5.43,MG,1643788158,NA
364,matara,5.9485,80.5353,30.72,50,8,4.57,LK,1643788172,Lantern Boutique Hotel by Reveal
421,ca mau,9.1769,105.1500,32.60,44,5,6.65,VN,1643787903,Khách Sạn Mường Thanh Luxury Cà Mau


In [115]:

hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,port elizabeth,-33.9180,25.5701,31.16,58,0,1.03,ZA,1643787981,Island Way Villa
43,busselton,-33.6500,115.3333,32.25,28,0,2.01,AU,1643788020,Bayview Geographe Resort
140,east london,-33.0153,27.9116,31.52,58,0,1.03,ZA,1643787987,Blue Lagoon Hotel
203,geraldton,-28.7667,114.6000,42.23,13,0,6.69,AU,1643788016,Ocean Centre Hotel
364,matara,5.9485,80.5353,30.72,50,8,4.57,LK,1643788172,Lantern Boutique Hotel by Reveal
421,ca mau,9.1769,105.1500,32.60,44,5,6.65,VN,1643787903,Khách Sạn Mường Thanh Luxury Cà Mau


In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [118]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))